In [2]:
# fitting model using SVM
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import seaborn as sns
# run unzip filename in terminal
import zipfile
import json
import os
pd.set_option('display.max_columns', None)

In [8]:
# load data files
matches_df = pd.read_json("/workspaces/project-9/sample_data/sample_data/d2_250_1_matches.jsonl", lines=True)
timelines_df = pd.read_json("/workspaces/project-9/sample_data/sample_data/d2_250_1_timelines.jsonl", lines=True)
print(matches_df['metadata'].head())

0    {'dataVersion': '2', 'matchId': 'NA1_542195455...
1    {'dataVersion': '2', 'matchId': 'NA1_541777086...
2    {'dataVersion': '2', 'matchId': 'NA1_542134877...
3    {'dataVersion': '2', 'matchId': 'NA1_542030874...
4    {'dataVersion': '2', 'matchId': 'NA1_542009850...
Name: metadata, dtype: object


In [ ]:
# convert data file contents into numbers so SVM can understand

In [ ]:
# flatten columns and add to dfs to fit to SVM model
matches_metadata_df = pd.json_normalize(matches_df["metadata"])
matches_info_df = pd.json_normalize(matches_df["info"])
matches_new_df = pd.concat([matches_df.drop(["metadata", "info"], axis=1),
                            matches_metadata_df,
                            matches_info_df], axis=1)
timelines_metadata_df = pd.json_normalize(timelines_df["metadata"])
timelines_info_df = pd.json_normalize(timelines_df["info"])
timelines_new_df = pd.concat([timelines_df.drop(["metadata", "info"], axis=1),
                            timelines_metadata_df,
                            timelines_info_df], axis=1)

dataVersion             0
matchId                 0
participants            0
endOfGameResult         0
gameCreation            0
gameDuration            0
gameEndTimestamp        0
gameId                  0
gameMode                0
gameName                0
gameStartTimestamp      0
gameType                0
gameVersion             0
mapId                   0
participants            0
platformId              0
queueId                 0
teams                   0
tournamentCode          0
gameModeMutators      223
dtype: int64
dataVersion        0
matchId            0
participants       0
endOfGameResult    0
frameInterval      0
frames             0
gameId             0
participants       0
dtype: int64


In [ ]:
# remove rows that contains NaN value(s)
matches_new_df = matches_new_df.dropna()
timelines_new_df = timelines_new_df.dropna()